In [3]:
###FEATURE EXTRACTION

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
#Getting the data
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

IMAGE_SIZE = (224,224)
BATCH_SIZE = 64

train_ds = train_gen.flow_from_directory(train_dir,
                                         batch_size=BATCH_SIZE,
                                         target_size=IMAGE_SIZE,
                                         class_mode="categorical")
test_ds = test_gen.flow_from_directory(test_dir,
                                      batch_size=BATCH_SIZE,
                                      target_size=IMAGE_SIZE,
                                      class_mode="categorical")

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [5]:
#callbacks
#tensorboard
import datetime

def create_tensorboard_callback(dir_name, exp_name):
    log_dir = dir_name + "/" + exp_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
    print("Saving TensorBoard log files to", logdir)
    return tensorboard_callback

In [11]:
#Creating models using tensorflow hub
import tensorflow_hub as hub
from tensorflow.keras import layers

def create_model(model_url, num_classes=10):
    """
    Takes a tf hub url and creates a Sequential model with it
    """
    feature_extractor_layer = hub.KerasLayer(model_url, 
                                             trainable=False,
                                            input_shape=IMAGE_SIZE+(3,))
    model = tf.keras.Sequential([
        feature_extractor_layer,
        layers.Dense(num_classes, activation="softmax")
    ])
    
    return model

In [35]:
efficientnet_v2_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b0/classification/2"

mobilenet_v3_url = "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5"

efficientnet_model = create_model(efficientnet_v2_url)

mobilenet_model = create_model(mobilenet_v3_url)

In [36]:
mobilenet_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=["accuracy"])

In [37]:
history_mobilenet = mobilenet_model.fit(train_ds,
                                             validation_data=test_ds,
                                             epochs=10,
                                             steps_per_epoch=len(train_ds),
                                             validation_steps=len(test_ds))

Epoch 1/10


2023-09-10 16:14:52.085517: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - ETA: 0s - loss: 2.0988 - accuracy: 0.2667

2023-09-10 16:14:58.254208: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 17s 1s/step - loss: 2.0988 - accuracy: 0.2667 - val_loss: 1.6449 - val_accuracy: 0.5420
Epoch 2/10
12/12 [==============================] - 7s 616ms/step - loss: 1.3963 - accuracy: 0.6573 - val_loss: 1.1647 - val_accuracy: 0.7228
Epoch 3/10
12/12 [==============================] - 6s 568ms/step - loss: 1.0128 - accuracy: 0.7933 - val_loss: 0.9000 - val_accuracy: 0.7800
Epoch 4/10
12/12 [==============================] - 7s 597ms/step - loss: 0.7883 - accuracy: 0.8387 - val_loss: 0.7569 - val_accuracy: 0.8168
Epoch 5/10
12/12 [==============================] - 6s 561ms/step - loss: 0.6599 - accuracy: 0.8587 - val_loss: 0.6747 - val_accuracy: 0.8268
Epoch 6/10
12/12 [==============================] - 6s 536ms/step - loss: 0.5730 - accuracy: 0.8747 - val_loss: 0.6255 - val_accuracy: 0.8356
Epoch 7/10
12/12 [==============================] - 6s 559ms/step - loss: 0.5060 - accuracy: 0.8933 - val_loss: 0.5898 - val_accuracy: 0.8416
Epoch 8/10
12/12 [=

In [38]:
mobilenet_model.evaluate(test_ds)

40/40 [==============================] - 5s 115ms/step - loss: 0.5270 - accuracy: 0.8488


[0.5270245671272278, 0.848800003528595]

In [23]:
efficientnet_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                          optimizer=tf.keras.optimizers.Adam(),
                          metrics=["accuracy"])
history_efficientnet = efficientnet_model.fit(train_ds,
                                             validation_data=test_ds,
                                             epochs=20,
                                             steps_per_epoch=len(train_ds),
                                             validation_steps=len(test_ds))

Epoch 1/20


2023-09-07 18:25:57.816710: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - ETA: 0s - loss: 428.8989 - accuracy: 0.1733

2023-09-07 18:26:02.646068: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 13s 1s/step - loss: 428.8989 - accuracy: 0.1733 - val_loss: 400.4788 - val_accuracy: 0.1904
Epoch 2/20
12/12 [==============================] - 7s 576ms/step - loss: 259.8909 - accuracy: 0.2080 - val_loss: 235.5818 - val_accuracy: 0.3180
Epoch 3/20
12/12 [==============================] - 6s 567ms/step - loss: 165.4703 - accuracy: 0.2733 - val_loss: 70.7217 - val_accuracy: 0.2624
Epoch 4/20
12/12 [==============================] - 6s 544ms/step - loss: 58.3142 - accuracy: 0.4280 - val_loss: 37.2986 - val_accuracy: 0.5712
Epoch 5/20
12/12 [==============================] - 7s 583ms/step - loss: 29.4950 - accuracy: 0.5347 - val_loss: 26.8800 - val_accuracy: 0.6756
Epoch 6/20
12/12 [==============================] - 7s 571ms/step - loss: 17.8022 - accuracy: 0.6307 - val_loss: 7.4949 - val_accuracy: 0.7192
Epoch 7/20
12/12 [==============================] - 6s 544ms/step - loss: 9.1637 - accuracy: 0.7227 - val_loss: 11.1579 - val_accuracy: 0.7496
Ep

In [24]:
efficientnet_model.evaluate(test_ds)

40/40 [==============================] - 5s 118ms/step - loss: 9.2386 - accuracy: 0.6796


[9.238638877868652, 0.6796000003814697]

In [40]:
###FINE TUNING THE MODEL

In [41]:
#importing helper functions
from helper_functions import *